In [75]:
import numpy as np 
import pandas as pd 
import json
import re

In [76]:
train_df = pd.read_json('clean_train.json')
test_df = pd.read_json('clean_test.json')
train_df

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,...,email_count,building_id_exist,date,month,day_of_week,hour,interest_num,pop_feature_counts,manager_type,building_type
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,1466754864000,brand new 3 bedroom 15 bath apartmentenjoy fo...,Metropolitan Avenue,,40.7145,7211212,-73.9425,...,0,True,2016-06-24,6,Friday,7,2,0,3,4
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,1465733967000,,Columbus Avenue,|doorman|elevator|fitness center|cats allowed|...,40.7947,7150865,-73.9667,...,0,True,2016-06-12,6,Sunday,12,3,5,3,3
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,1460863601000,top top west village location beautiful prewa...,W 13 Street,|laundry in building|dishwasher|hardwood floor...,40.7388,6887163,-74.0018,...,0,True,2016-04-17,4,Sunday,3,1,3,4,4
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,1460946122000,build amenity garage garden fitness room laun...,East 49th Street,|hardwood floors|no fee,40.7539,6888711,-73.9677,...,0,True,2016-04-18,4,Monday,2,3,2,3,3
100013,1.0,4,0,1461807161000,beautifully renovate 3 bedroom flex 4 bedroom...,West 143rd Street,|prewar,40.8241,6934781,-73.9493,...,0,False,2016-04-28,4,Thursday,1,3,1,3,3
100014,2.0,4,38a913e46c94a7f46ddf19b756a9640c,1461039887000,,West 18th Street,,40.7429,6894514,-74.0028,...,0,True,2016-04-19,4,Tuesday,4,2,0,3,4
100016,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,1461727196000,stun unit great location lot natural light cl...,West 107th Street,|prewar|elevator|dogs allowed|cats allowed|low...,40.8012,6930771,-73.9660,...,2,True,2016-04-27,4,Wednesday,3,3,8,3,3
100020,2.0,1,0372927bcb6a0949613ef5bf893bbac7,1460527302000,huge sunny plenty light 1 bed2 bath offer bra...,West 21st Street,|doorman|elevator|prewar|terrace|laundry in un...,40.7427,6867392,-73.9957,...,0,True,2016-04-13,4,Wednesday,6,3,8,4,3
100026,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,1461119795000,pa website_redacted,Hamilton Terrace,|cats allowed|dogs allowed|elevator|laundry in...,40.8234,6898799,-73.9457,...,0,True,2016-04-20,4,Wednesday,2,2,4,4,4
100027,2.0,4,0,1459565895000,spacious four bedroom every bedroom able fit ...,522 E 11th,|dishwasher|hardwood floors,40.7278,6814332,-73.9808,...,0,False,2016-04-02,4,Saturday,2,3,2,4,3


In [77]:
from sklearn.model_selection import train_test_split
X = train_df
y = train_df['interest_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [78]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def transform(self, df, y=None):
        return df[self.column]
    
    def fit(self, df, y=None):
        return self

In [79]:
class ColumnAttribute(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def transform(self, df, y=None):
        data=df.apply(lambda x: x)
        return np.transpose(np.matrix(data))
    
    def fit(self, df, y=None):
        return self

In [80]:
from collections import Counter
from operator import itemgetter

def popular_featrues(feature_column):
        all_features = ''
        counter = Counter()
        for text in feature_column:
            text = text.split('|')
            all_features = ''.join([all_features, ','.join(text)])
        all_features = all_features.strip(',').split(',')
        #all_features = [item for item in all_features if item != '']
        counter = Counter(all_features)
        popular_features = [k for k, v in sorted(counter.items(), key=itemgetter(1), reverse=True)][:50]
        return popular_features

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [82]:
'''
text_pipe = Pipeline([('vect', CountVectorizer(max_df= 0.75, max_features= None, ngram_range= (1, 2))),
                      ('tfidf', TfidfTransformer(norm= 'l2', use_idf= False))])
                      
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier

pipeline = Pipeline([
    ('feats', FeatureUnion([
        ('text_pipe', text_pipe), 
        ('upper', upperchars()),
        ('repeating', repeating()),
        ('exclaim', exclamation()),
        ('colon', colon())
    ])),
    ('clf', SGDClassifier(alpha= 1e-05, max_iter= 50, penalty= 'elasticnet'))  # classifier
])
'''

"\ntext_pipe = Pipeline([('vect', CountVectorizer(max_df= 0.75, max_features= None, ngram_range= (1, 2))),\n                      ('tfidf', TfidfTransformer(norm= 'l2', use_idf= False))])\n                      \nfrom sklearn.pipeline import Pipeline, FeatureUnion\nfrom sklearn.linear_model import SGDClassifier\n\npipeline = Pipeline([\n    ('feats', FeatureUnion([\n        ('text_pipe', text_pipe), \n        ('upper', upperchars()),\n        ('repeating', repeating()),\n        ('exclaim', exclamation()),\n        ('colon', colon())\n    ])),\n    ('clf', SGDClassifier(alpha= 1e-05, max_iter= 50, penalty= 'elasticnet'))  # classifier\n])\n"

In [83]:
model = Pipeline([
    ('features', FeatureUnion([
            ('feature1', Pipeline([
                ('selector', ItemSelector(column='tele_count')),
                ('tele_count', ColumnAttribute())
            ])),
            ('feature2', Pipeline([
                ('selector', ItemSelector(column='web_count')),
                ('web_count', ColumnAttribute())
            ])),
            ('feature3', Pipeline([
                ('selector', ItemSelector(column='email_count')),
                ('email_count', ColumnAttribute())
            ])),
            ('feature4', Pipeline([
                ('selector', ItemSelector(column='price')),
                ('price', ColumnAttribute())
            ])),
            ('feature5', Pipeline([
                ('selector', ItemSelector(column='bedrooms')),
                ('bedrooms', ColumnAttribute())
            ])),
            ('feature6', Pipeline([
                ('selector', ItemSelector(column='des_length')),
                ('des_length', ColumnAttribute())
            ])),
            ('feature7', Pipeline([
                ('selector', ItemSelector(column='num_features')),
                ('num_features', ColumnAttribute())
            ])),
            ('feature8', Pipeline([
                ('selector', ItemSelector(column='latitude')),
                ('latitude', ColumnAttribute())
            ])),
            ('feature9', Pipeline([
                ('selector', ItemSelector(column='longitude')),
                ('longitude', ColumnAttribute())
            ])),
            ('feature10', Pipeline([
                ('selector', ItemSelector(column='manager_type')),
                ('manager_type', ColumnAttribute())
            ])),
            ('feature11', Pipeline([
                ('selector', ItemSelector(column='building_type')),
                ('building_type', ColumnAttribute())
            ])),
            ('feature12', Pipeline([
                ('selector', ItemSelector(column='pop_feature_counts')),
                ('building_type', ColumnAttribute())
            ])),
     ])),
     ('clf', LogisticRegression())
])
print(1)

1


In [84]:
'''
('word_features1', Pipeline([
        ('selector', ItemSelector(column='description')),
        ('vect', CountVectorizer(max_df=0.75, ngram_range=(1, 4))), 
        ('tfidf', TfidfTransformer(use_idf = True)),
])),
('word_features2', Pipeline([
        ('selector', ItemSelector(column='features')),
        ('vect', CountVectorizer(max_df=0.75, ngram_range=(1, 4))), 
        ('tfidf', TfidfTransformer(use_idf = True)),
])),
'''

"\n('word_features1', Pipeline([\n        ('selector', ItemSelector(column='description')),\n        ('vect', CountVectorizer(max_df=0.75, ngram_range=(1, 4))), \n        ('tfidf', TfidfTransformer(use_idf = True)),\n])),\n('word_features2', Pipeline([\n        ('selector', ItemSelector(column='features')),\n        ('vect', CountVectorizer(max_df=0.75, ngram_range=(1, 4))), \n        ('tfidf', TfidfTransformer(use_idf = True)),\n])),\n"

In [85]:
# Fit the model
model.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
pred = model.predict(X_test)
test = y_test

accuracy_score(test, pred)

/Users/chenlu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/chenlu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7257868431716872

In [89]:
from sklearn.metrics import classification_report, log_loss
prediction = model.predict(X_test)
prediction_proba = model.predict_proba(X_test)
print(classification_report(y_test, prediction))
print(log_loss(y_test, prediction_proba))

              precision    recall  f1-score   support

        high       0.20      0.00      0.00      1086
         low       0.80      0.90      0.85     10404
      medium       0.45      0.42      0.43      3316

   micro avg       0.73      0.73      0.73     14806
   macro avg       0.48      0.44      0.43     14806
weighted avg       0.68      0.73      0.69     14806

0.6130610149122084


In [90]:
test_df['interet_level'] = model.predict(test_df)

In [91]:
test_prediction = model.predict(test_df)
prediction_proba = model.predict_proba(test_df)
test_prediction

array(['low', 'low', 'low', ..., 'low', 'low', 'low'], dtype=object)

In [92]:
prediction_proba

array([[0.12844993, 0.46529429, 0.40625578],
       [0.01705302, 0.90224281, 0.08070416],
       [0.05689743, 0.61480512, 0.32829744],
       ...,
       [0.06150535, 0.74567028, 0.19282437],
       [0.13800168, 0.60188614, 0.26011218],
       [0.01749142, 0.89826423, 0.08424435]])

In [93]:
labels2idx = {label: i for i, label in enumerate(model.classes_)}
labels2idx

{'high': 0, 'low': 1, 'medium': 2}

In [94]:
labels2idx = {label: i for i, label in enumerate(model.classes_)}
sub = pd.DataFrame()
sub["listing_id"] = test_df["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = prediction_proba[:, labels2idx[label]]

In [95]:
sub.to_csv("sub6.csv", index=False)